In [ ]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["kin4"]

Copyright **Jacob Martin and Paolo Raiteri**, January 2021

## Determination of the Rate Law \#4
You are studying the rate of decomposition of species `A` by monitoring its concentration with time. 
Collect enough data to determine the order of the reaction, the rate constant, and the half-life.

### Instructions:

- Use the slide bar(s) below to select the times at which you perform the measurement.
- Click `Perform measurement` to run the virtual experiment and collect the result.
- Click `Download CSV` to export the complete data set for all the experiments as a CSV file.
- Note that every time you `Restart laboratory` some parameters of the experiments may change.

In [ ]:
# define path to results.csv file
respath = os.path.join(os.getcwd(), "..", "results.csv")

# delete existing result file and setup rng
if os.path.exists(respath):
    os.remove(respath)

#random.seed(params["error"].get("seed", 0))
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

class system:
    def __init__(self, vol=0, conc=0, press=0):
        self.vol = vol
        self.conc = conc
        self.press = press

class data:
    def __init__(self, start=-1, error=0, label='none', units='pure', value=0,
                minval=-1, maxval=3):
        self.start = start
        self.minval = minval
        self.maxval = maxval
        self.error = error
        self.label = label
        self.units = units
        self.value = value
    
# Experiment setup (+ hidden paramters)
system = system()
def initialiseExperiment():
    global n
    global system 
    global columns_list
    global scatter
    
    scatter = 0.1
    
    n = []
    columns_list = []

    n.append(len(args)) # number of input adjustable parameters    
    n.append(len(result)) # number of results for the experiment

    for i in range(0, n[0]):
        columns_list.append(f"{args[i].label} [{args[i].units}]")
    for i in range(0, n[1]):
        columns_list.append(f"{result[i].label} [{result[i].units}]")

    # Random initial concentration
    system.conc = random.random()
 


In [ ]:
# Adjustable input parameters
def initialiseVariables():
    global logScale
    logScale = True
    global args
    args = []
    args.append(
        data(
            label = "Elapsed time",
            minval = 0,
            maxval = 5,
            start = 1.,
            units = "s",
            value = 0.
        )
    )

# Results
def initialiseResults():
    global result
    result = []
    result.append(
        data(
            label = "[A]",
            start = 0.,
            error = random.random() / 10.,
            units = "mol/L"
        )
    )

def measure():
    res = system.conc * (math.exp(-params["k"] * args[0].value.value))
    return res

initialiseVariables()


In [ ]:
out_P = ipw.Output()
out_L = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))
    
def calc(btn):
    out_P.clear_output()
    
    # Measurement result
    result[0].value = measure()
    
    # Random error
    result[0].error = result[0].value * scatter * (0.5 - random.random()) * 2
    
    # Output result
    out_R[0].value = f"{result[0].value + result[0].error:.3e}"

    # Read previous lines
    res = pd.read_csv(respath) 
    
    var_list = []
    for i in range(0, n[0]):
        var_list.append(args[i].value.value)
    for i in range(0, n[1]):
        var_list.append(result[i].value + result[i].error)
    
    # Append result
    res.loc[len(res)] = var_list
    res.to_csv(respath, index=False)
    with out_P:
        display(res.tail(50))

def reset(btn):
    if os.path.exists(respath):
        os.remove(respath)

    initialiseResults()
    initialiseExperiment()
    
    res = pd.DataFrame(columns=columns_list)
    res.to_csv(respath, index=False)
    with out_P:
        out_P.clear_output()
        display(res.tail(50))
    

# interactive buttons ---
btn_reset = ipw.Button(description="Restart Laboratory", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)

btn_calc = ipw.Button(description="Perform measurement", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)
# ---

reset(btn_reset)

rows = []
for i in range(0, n[0]):
    if logScale:
        args[i].value = ipw.FloatLogSlider(value=args[i].start, min=args[i].minval, max=args[i].maxval)
    else:
        args[i].value = ipw.FloatSlider(value=args[i].start, min=args[i].minval, max=args[i].maxval)
        
    rows.append(ipw.HBox([ipw.Label(value=f"{args[i].label} [{args[i].units}]:",
                                    layout=ipw.Layout(width="250px")),
                          args[i].value]))

out_R = []
for i in range(0, n[1]):
    out_R.append(ipw.Label(value=""))
    rows.append(ipw.HBox([ipw.Label(value=f"Measured {result[i].label} [{result[i].units}]:",
                                    layout=ipw.Layout(width="250px")),
                            out_R[i]]))

rows.append(ipw.HBox([btn_reset, btn_calc, out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)
